# AWS SageMaker and nomic-embed-text-v1.5
This notebook demos how to embed your documents using nomic-embed-text-v1.5 running on AWS SageMaker.

In [ ]:
!pip install nomic'[aws]'
!pip install numpy

In [1]:
import numpy as np

from nomic.aws.sagemaker import embed_text

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## SageMaker Configuration
Specify the Sagemaker endpoint you created here and the region the endpoint is hosted in.

In [2]:
endpoint_name = 'nomic-embed-endpoint'
region_name = 'us-east-2'

## Real Time Inference
Run `embed_text` for real time inference.
The method will return a nested list of floats. 

In [3]:
texts = [
    "This is a test", 
    "This is another test",
    "This is a third test",
    "The quick brown fox jumps over the lazy dog",
    "The quick brown fox jumps over the lazy dog",
    "The quick brown fox jumps over the lazy dog",
]

In [4]:
response = embed_text(texts, endpoint_name, region_name=region_name, batch_size=32)
embeddings = response["embeddings"]
np.array(embeddings).shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.13it/s]


(6, 768)

In [6]:
response = embed_text(texts, endpoint_name, region_name=region_name, batch_size=32, dimensionality=128, task_type="search_document")
embeddings = response["embeddings"]
np.array(embeddings).shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.64it/s]


(6, 128)

In [7]:
response = embed_text(texts, endpoint_name, region_name=region_name, batch_size=32, binary=True)
embeddings = response["embeddings"]
np.array(embeddings)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.29it/s]


array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

## Batch Transform

Nomic Embed on Sagemaker also supports [batch transform jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) over CSV files.
To run batch transform, make sure your csv contains rows of single column text without headers.

Optionally, in the first row, you can pass two columns: an integer value (valid options are 64, 128, 256, 512, 768) and a boolean. If the first row matches this format, these values will be treated as the dimensionality to return the embeddings in and whether to return binary embeddings or not. Otherwise, the first row will be treated as single column text as well (i.e. the first column will be cast to string and embedded).

For example, a valid input csv can look like:

```
128, True
Hello world
Hello world
Hello world
```

This will return three 128 dimension binary embeddings.

You can launch a batch transform job with the `nomic` package:

In [ ]:
from nomic.aws.sagemaker import batch_transform_text as batch_transform

batch_transform(
    s3_input_path="s3://batch-transform/input.csv",
    s3_output_path="s3://batch-transform/",
    region_name="us-east-2",
    arn='<INSERT MODEL PACKAGE ARN>',
    role='<INSERT SAGEMAKER EXECUTOR IAM ROLE ARN>',
    max_payload=6,
    n_instances=1
)